# Maryland Select LiDAR tiles

The project examines slope rasters derived from LiDAR to find RCH. Evidence of RCH is more often found in forested areas where ground has not been plowed or built on and so it may be useful to examine only liDAR tiles that contain forested areas.

## Inputs

### LiDAR tiles shapefile

This is from FEMA, projected to crs 26985:

lidar_tiles_path = "/content/drive/MyDrive/crane_maryland/lidar_tile_polygons/2012_fema_md_index_26985.shp"

### Land use: Forested

This is from the state of Maryland, filtered for only forested areas:

land_use_path = "/content/drive/MyDrive/crane_maryland/land_use/Maryland_Land_Use_Land_Cover_-_Land_Use_Land_Cover_2010_forest.shp"

## Outputs

A shapefile to contain LiDAR tiles that are in forested areas:

lidar_tiles_forested_polys_fp = "/content/drive/MyDrive/crane_maryland/polys/2012_fema_md_forested.shp"

## Method

Examine each LiDAR tile polygon and see if it overlaps the Geopandas Geoseries of forested polygons.

```
if (land_use_geoseries.overlaps(pred_poly).any()==True) or (land_use_geoseries.contains(pred_poly).any()==True) or (land_use_geoseries.within(pred_poly).any()==True):
```

## Notes

Using geoseries.overlap results in not including some LiDAR tile polygons that are entirely inside of a forested area. The overlap method must not work for that. https://geopandas.readthedocs.io/en/latest/docs/reference/api/geopandas.GeoSeries.overlaps.html "Geometries overlaps if they have more than one but not all points in common." Adding .contains and .within as well.


In [3]:
!pip install geopandas

     |████████████████████████████████| 1.0 MB 7.9 MB/s 
     |████████████████████████████████| 15.4 MB 77 kB/s 
     |████████████████████████████████| 6.3 MB 60.1 MB/s 


In [ ]:
import os
import geopandas as gpd

lidar_tiles_path = "/content/drive/MyDrive/crane_maryland/lidar_tile_polygons/2012_fema_md_index_26985.shp"
land_use_path = "/content/drive/MyDrive/crane_maryland/land_use/Maryland_Land_Use_Land_Cover_-_Land_Use_Land_Cover_2010_forest.shp"
lidar_tiles_forested_polys_fp = "/content/drive/MyDrive/crane_maryland/polys/2012_fema_md_forested.shp"

# Create an empty geopandas GeoDataFrame for duplicates
lidar_tiles_forested_polys_df = gpd.GeoDataFrame()
lidar_tiles_forested_polys_df['Index'] = None
lidar_tiles_forested_polys_df['Name'] = None
lidar_tiles_forested_polys_df['URL'] = None
lidar_tiles_forested_polys_df['geometry'] = None

lidar_tiles_forested_polys_df.crs = ('EPSG:26985')

if os.path.exists(lidar_tiles_path) and os.path.exists(land_use_path):
    lidar_tiles_polys = gpd.read_file(lidar_tiles_path)
    land_use_polys = gpd.read_file(land_use_path)
    print("crs", lidar_tiles_polys.crs,land_use_polys.crs )
    print(type(land_use_polys))
    print(list(land_use_polys))
    print("lidar_tiles_polys",lidar_tiles_polys.shape, "land_use_polys",land_use_polys.shape)
    land_use_geoseries = land_use_polys['geometry']
    print(type(land_use_geoseries))
    # crs for Maryland
    area_crs = 26985
    matched_tiles_polys = list()    
    lidar_tiles_polys.to_crs(area_crs)
    land_use_polys.to_crs(area_crs)

    for index, row in lidar_tiles_polys.iterrows():
       #print("row",row)

       pred_poly = row[3]
       #print("-----")
       #print("pred_poly",type(pred_poly), pred_poly)
       #print("shape",land_use_geoseries.shape)
       #print("++++++++++++")

       #print(land_use_geoseries.overlaps(pred_poly).any())
       if (land_use_geoseries.overlaps(pred_poly).any()==True) or (land_use_geoseries.contains(pred_poly).any()==True) or (land_use_geoseries.within(pred_poly).any()==True):

           print("match", row[0])  
           lidar_tiles_forested_polys_df = lidar_tiles_forested_polys_df.append(row, ignore_index=True)
       else:
           print("    no match", row[0])  
          

if not lidar_tiles_forested_polys_df.empty:
    lidar_tiles_forested_polys_df.to_file(lidar_tiles_forested_polys_fp)
    print("Total forested tiles:", len(lidar_tiles_forested_polys_df))
else:
    print("empty")

In [ ]:
print(lidar_tiles_forested_polys_df)